In [1]:
#libraries we are going to use
import numpy as np # linear algebra
import pandas as pd # data processing
from plotly.offline import iplot, init_notebook_mode #visualization
import plotly.express as px #visualization
from plotly.subplots import make_subplots #visualization
import plotly.graph_objects as go #visualization
import matplotlib.pyplot as plt #visualization
import seaborn as sns #visualization
from sklearn import preprocessing #label encoder 
from sklearn.model_selection import train_test_split #split data
from sklearn.linear_model import LogisticRegression #model
from sklearn.model_selection import cross_val_score #cross validation
from sklearn.metrics import classification_report #reports 
from sklearn import metrics #confusion metrics
from sklearn.model_selection import GridSearchCV #get best parameters

In [2]:
#import the filenames
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
sellout = pd.read_excel('/content/sellout.xlsx')

In [6]:
sellout.head()

,Month,Distributor Name,Internal transactions between DT,Customer Name,Legal Entity,City_1,Model / Product Number,Detailed Sell Out Units,Whether the product line counts rebates,Product Line,...,Unnamed: 22,менеджер,OOO ST NETVORKING,OOO MERLION,TradeicsBelPlus LLC,Netlab,"TORUS,.ltd",Marvel,OCS-Center Ltd.,Treolan
0,1,OOO ST NETVORKING,N,3С ГРУПП ПРОЕКТ,"ООО ""3С ГРУПП ПРОЕКТ""",Москва г,DH-XVR5104HS-4KL-I3,1.0,Y,HDXVR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,OOO ST NETVORKING,N,3С ГРУПП ПРОЕКТ,"ООО ""3С ГРУПП ПРОЕКТ""",Москва г,DH-IPC-HFW2431SP-S-0280B,10.0,Y,IPC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,OOO ST NETVORKING,N,3С ГРУПП ПРОЕКТ,"ООО ""3С ГРУПП ПРОЕКТ""",Москва г,DH-HAC-HDW1231TP-Z-A,17.0,Y,HDCVI Camera,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,OOO ST NETVORKING,N,3С ГРУПП ПРОЕКТ,"ООО ""3С ГРУПП ПРОЕКТ""",Москва г,DHI-NKB1000,2.0,Y,Decoders,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,OOO ST NETVORKING,N,3С ГРУПП ПРОЕКТ,"ООО ""3С ГРУПП ПРОЕКТ""",Москва г,DH-IPC-HFW2431SP-S-0360B,4.0,Y,IPC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
sellout.columns

Index(['Month', 'Distributor Name', 'Internal transactions between DT',
       'Customer Name', 'Legal Entity', 'City_1', 'Model / Product Number',
       'Detailed Sell Out Units', 'Whether the product line counts rebates',
       'Product Line', 'MSRP', 'MRSP TotalSellout, RUB',
       'calculation achievement', 'calculation rebate', 'City', 'Region',
       'Region.1', 'Status', 'sub-Status',
       'Whether the DT counts rebates (only for DPP)', 'Partner ID',
       'System Code', 'Unnamed: 22', 'менеджер', 'OOO ST NETVORKING',
       'OOO MERLION', 'TradeicsBelPlus LLC', 'Netlab', 'TORUS,.ltd', 'Marvel',
       'OCS-Center Ltd.', 'Treolan'],
      dtype='object')

In [18]:
bg = pd.DataFrame({'col1':[1,1,2,3,4,5]})
bg

,col1
0,1
1,1
2,2
3,3
4,4
5,5


In [19]:
bg['col1'] = bg['col1'].replace([1,2,3,4,5],['one','two','three','four','five'])
bg

,col1
0,one
1,one
2,two
3,three
4,four
5,five


In [27]:
sellout['Legal Entity'] = sellout['Legal Entity'].replace(['ИП Хоршунова Анна Сергеевна',' ООО "Компания Айпера"','ООО "Компания Айпера"','ООО "Торговый Дом ТИНКО"','ИП Журавлев Геннадий Владимирович','ООО "ТК "МИР БЕЗОПАСНОСТИ"','ООО МИР БЕЗОПАСНОСТИ','ООО ФСБ+','ООО " КМ-ЮГ " ','ООО ""Линдекс технолоджис СПб""'],['"ИП Хоршунова А.С."','ООО "АЙПЕРА','ООО "АЙПЕРА','ООО "ТД ТИНКО"','ИП Журавлев Г.В.','ООО "МИР БЕЗОПАСНОСТИ"','ООО "МИР БЕЗОПАСНОСТИ"','ООО "ФБС+"','ООО "КМ-Юг"','ООО "Линдекс технолоджис СПб"'])

In [29]:
sellout['Legal Entity'] = sellout['Legal Entity'].replace(['ООО "Линдекс технолоджис Юг"','СБ Монтаж','ООО «ГРИМИХС»','ООО "Аметист" (СПб)','ЗАО «Теко»','ООО «Альтерико»','ООО Техносервис ','ООО "ТК "РУТЕК"','ООО «Рифт»','"ООО Инжиниринг Электрик"'],['ООО "Линдекс технолоджис СПб"','ООО "СБ-Монтаж"','ООО "ГРИМИХС-ЮГ"','ООО "Аметист"','ООО "Теко"','ООО "АЛЬТЕРИКО"','ООО "Техносервис"','ООО "ТК РУТЕК"','ООО "РИФТ"','ООО "Инжиниринг электрик"'])

In [30]:
sellout['Legal Entity'] = sellout['Legal Entity'].replace(['ООО "ТД "Русичи" ','ООО ТД "Русичи"','ООО "4С Групп Проект"','ООО "5С Групп Проект"','ООО "Сэйфти"','ООО ТД «ЭТОС»','ООО " НТЦ "СТАРКО"','ООО «НТЦ «СтарКо»','ООО "Дозор"','ООО «ТК Аврора»'],['ООО "ТД Русичи" ','ООО "ТД Русичи" ','ООО "3С ГРУПП ПРОЕКТ"','ООО "3С ГРУПП ПРОЕКТ"','ООО "ТД Сэйфти"','ООО ТД "ЭТОС" ','ООО "НТЦ СТАРКО"','ООО "НТЦ СТАРКО"','ООО "Дозор"','ООО "ТК АВРОРА"'])


In [31]:
sellout['Legal Entity'] = sellout['Legal Entity'].replace(['ООО М-Инвест','ООО "КИТ" Новороссийск','ООО «Диамант+»','ООО Проф-Сервис ','ООО Арсенал Партнер','"ООО "ТМК Периметр+"','ООО "ТМК "Периметр"','ООО "ФорСек"','ООО Пролайн Системы Безопасности','ООО Тайм Секьюрити '],['ООО "М-Инвест"','ООО "КИТ"','ООО "Диамант"','ООО "Проф-Сервис"','ООО "Арсенал Партнер"','ООО "ТМК Периметр"','ООО "ТМК Периметр"','ООО "Форсек" ','ООО "ПРОЛАЙН Системы Безопасности"','ООО "Тайм Секьюрити"'])

In [32]:
sellout['Legal Entity'] = sellout['Legal Entity'].replace(['ООО «Открытые технологии»','Открытые технологии ООО','Кверт ООО','ООО "Южный Форпост"','Аэротел','ООО "КОМПАНИЯ "7000ЛВ"','ООО "Компания"7000лв"','ООО "АС-ВИДЕО" ','ООО "Амиком Трейдинг"','ДИСКОНТ-ВИДЕО М'],['ООО "Открытые технологии"','ООО "Открытые технологии"','ООО "Кверт"','ООО "ФОРПОСТ ЮЖНЫЙ"','ООО "Аэротел"','ООО "Компания 7000лв"','ООО "Компания 7000лв"','ООО "АС-ВИДЕО"','ООО "АМИКОМ ТРЕЙД"','ООО "ДИСКОНТ-ВИДЕО М"'])

In [33]:
sellout['Legal Entity'] = sellout['Legal Entity'].replace(['ООО «Защита»   ','ООО "СТ-Инвест"','ООО "Технологии Безопасности"','ООО "Эффорт Групп"','ООО "Эффорт Северо-Запад"'],['ООО  "ЗАЩИТА"','ООО "СТ ИНВЕСТ"','ООО "Техника безопасности"','ООО "Эффорт"','ООО "Эффорт"'])

In [35]:
sellout.to_csv('sellout_new.csv')